*This analysis is preliminary and based on a toy dataset (n=2), intended solely to validate the inference and analysis pipeline. No performance conclusions are drawn.*

In [63]:
import os, sys

PROJECT_ROOT = "/content/nlp-finetuning-bert"
os.chdir(PROJECT_ROOT)
sys.path.append(PROJECT_ROOT)

print("CWD:", os.getcwd())
assert os.path.exists("data/predictions_with_analysis.csv"), "Brak data/predictions_with_analysis.csv"

CWD: /content/nlp-finetuning-bert


In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import (
    classification_report,
    confusion_matrix
)

In [65]:
df = pd.read_csv("data/predictions_with_analysis.csv")

df.head(), df.shape

(                                                text     label  confidence  \
 0  Psilocybin induces rapid structural plasticity...  POSITIVE      0.8810   
 1  No significant neuroplastic effects were obser...  NEGATIVE      0.5326   
 
   label_thresholded  
 0          POSITIVE  
 1         UNCERTAIN  ,
 (2, 4))

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   text        2 non-null      object 
 1   label       2 non-null      object 
 2   confidence  2 non-null      float64
dtypes: float64(1), object(2)
memory usage: 180.0+ bytes


In [61]:
df.isna().sum()

,0
text,0
label,0
confidence,0


In [68]:
df["label"].value_counts()

,count
label,
POSITIVE,1
NEGATIVE,1


In [67]:
df["label_thresholded"].value_counts()

,count
label_thresholded,
POSITIVE,1
UNCERTAIN,1


In [70]:
print(
    classification_report(
        df["label"],
        df["label_thresholded"],
        labels=["NEGATIVE", "POSITIVE"],
        zero_division=0
    )
)


confusion_matrix(df["label"], df["label_thresholded"])

              precision    recall  f1-score   support

    NEGATIVE       0.00      0.00      0.00         1
    POSITIVE       1.00      1.00      1.00         1

   micro avg       1.00      0.50      0.67         2
   macro avg       0.50      0.50      0.50         2
weighted avg       0.50      0.50      0.50         2



array([[0, 0, 1],
       [0, 1, 0],
       [0, 0, 0]])

At the current stage, the model exhibits a tendency to route low-confidence negative samples into the UNCERTAIN category under a conservative threshold (0.7), which is expected behavior given the design of the inference logic.

In [78]:
!git add notebooks/error_analysis.ipynb
!git commit -m "Add preliminary error analysis (pipeline validation)"
!git push

[main b9e7baa] Add preliminary error analysis (pipeline validation)
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite notebooks/error_analysis.ipynb (97%)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 802 bytes | 802.00 KiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
remote: This repository moved. Please use the new location:
remote:   https://github.com/Masesun/nlp-finetuning-bert.git
To https://github.com/masesun/nlp-finetuning-bert.git
   e68bf3c..b9e7baa  main -> main


In [77]:
!cp "/content/drive/MyDrive/Colab Notebooks/nlp-finetuning-bert.ipynb" \
   "/content/nlp-finetuning-bert/notebooks/error_analysis.ipynb"